# Working with datetime in Pandas DataFrame

<img src="images/datetimememe.webp">

In [1]:
import pandas as pd
import numpy as np

### 1. Convert strings to datetime

#### Default arguments

In [2]:
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})


In [3]:
df

,date,value
0,3/10/2000,2
1,3/11/2000,3
2,3/12/2000,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   value   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
 1   value   3 non-null      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 176.0 bytes


In [7]:
df

,date,value
0,2000-03-10,2
1,2000-03-11,3
2,2000-03-12,4


#### Day first format

In [8]:
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [9]:
df

,date,value
0,2000-10-03,2
1,2000-11-03,3
2,2000-12-03,4


#### Custom format 

Ref - https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [14]:
df = pd.DataFrame({'date': ['2016-6-10 20:30:0', 
                            '2016-7-1 19:45:30', 
                            '2013-10-12 4:5:1'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], format="%Y-%d-%m %H:%M:%S")

In [15]:
df

,date,value
0,2016-10-06 20:30:00,2
1,2016-01-07 19:45:30,3
2,2013-12-10 04:05:01,4


#### Speedup parsing with infer_datetime_format

In [17]:
df = pd.DataFrame({'date': ['3/11/2000', '3/12/2000', '3/13/2000'] * 1000 })
df.head()

,date
0,3/11/2000
1,3/12/2000
2,3/13/2000
3,3/11/2000
4,3/12/2000


In [18]:
%timeit pd.to_datetime(df['date'], infer_datetime_format=True)

449 µs ± 8.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%timeit pd.to_datetime(df['date'], infer_datetime_format=False)

434 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Handle parsing error

In [20]:
df = pd.DataFrame({'date': ['3/10/2000', 'a/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})


In [21]:
df

,date,value
0,3/10/2000,2
1,a/11/2000,3
2,3/12/2000,4


In [22]:
df['date'] = pd.to_datetime(df['date'])

ParserError: Unknown string format: a/11/2000

In [23]:
df['date'] = pd.to_datetime(df['date'], errors='ignore')
df

,date,value
0,3/10/2000,2
1,a/11/2000,3
2,3/12/2000,4


In [24]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df

,date,value
0,2000-03-10,2
1,NaT,3
2,2000-03-12,4


### 2. Assemble a datetime from multiple columns

In [25]:
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})


In [26]:
df

,year,month,day
0,2015,2,4
1,2016,3,5


In [27]:

df['date'] = pd.to_datetime(df)

In [28]:
df

,year,month,day,date
0,2015,2,4,2015-02-04
1,2016,3,5,2016-03-05


### 3. Get year, month and day

In [29]:
df = pd.DataFrame({'name': ['Tom', 'Andy', 'Lucas'],
                 'DoB': ['08-05-1997', '04-28-1996', '12-16-1995']})


In [30]:
df

,name,DoB
0,Tom,08-05-1997
1,Andy,04-28-1996
2,Lucas,12-16-1995


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3 non-null      object
 1   DoB     3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [32]:
df['DoB'] = pd.to_datetime(df['DoB'])

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   name    3 non-null      object        
 1   DoB     3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 176.0+ bytes


In [37]:
df['year']= df['DoB'].dt.year
df['month']= df['DoB'].dt.month
df['day']= df['DoB'].dt.day
df

,name,DoB,year,month,day
0,Tom,1997-08-05,1997,8,5
1,Andy,1996-04-28,1996,4,28
2,Lucas,1995-12-16,1995,12,16


### 4. Get the week of year, the day of week and leap year

In [38]:
df['week_of_year'] = df['DoB'].dt.week
df['day_of_week'] = df['DoB'].dt.dayofweek
df['is_leap_year'] = df['DoB'].dt.is_leap_year
df

/var/folders/0n/nb074wzd4kl1nmdvkbdnpgym0000gn/T/ipykernel_11495/3969492849.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df['DoB'].dt.week


,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year
0,Tom,1997-08-05,1997,8,5,32,1,False
1,Andy,1996-04-28,1996,4,28,17,6,True
2,Lucas,1995-12-16,1995,12,16,50,5,False


In [39]:
df['DoB'].dt.weekday

0    1
1    6
2    5
Name: DoB, dtype: int64

In [40]:
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 
df['day_of_week_name']=df['DoB'].dt.weekday.map(dw_mapping)
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday


### 5. Get the age from the date of birth

In [44]:
pd.to_datetime('today').year - df['DoB'].dt.year

0    26
1    27
2    28
Name: DoB, dtype: int64

0    1997
1    1996
2    1995
Name: DoB, dtype: int64

In [45]:
today = pd.to_datetime('today')
df['age'] = today.year - df['DoB'].dt.year

df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,26
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,27
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,28


In [52]:
# Year difference
today = pd.to_datetime('today')
diff_y = today.year - df['DoB'].dt.year


# Haven't had birthday
b_md = df['DoB'].apply(lambda x: (x.month,x.day) )
no_birthday = b_md > (today.month,today.day)

df['age'] = diff_y - no_birthday
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,25
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,26
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,27


In [51]:
no_birthday

0    True
1    True
2    True
Name: DoB, dtype: bool

### 6. Improve performance by setting date column as the index 

In [53]:
df = pd.read_csv('data/city_sales.csv',parse_dates=['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795144 entries, 0 to 1795143
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   date    datetime64[ns]
 1   num     int64         
 2   city    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 41.1+ MB


In [54]:
df = df.set_index(['date'])

In [55]:
df

,num,city
date,,
2015-01-01 09:00:00,4,London
2015-01-01 09:01:00,4,London
2015-01-01 09:02:00,3,London
2015-01-01 09:03:00,3,London
2015-01-01 09:04:00,3,London
...,...,...
2019-01-31 15:56:00,3,Cambridge
2019-01-31 15:57:00,3,Cambridge
2019-01-31 15:58:00,3,Cambridge


### 7. Select data with a specific year and perform aggregation

In [56]:
df.loc['2018']

,num,city
date,,
2018-01-01 09:00:00,2,London
2018-01-01 09:01:00,1,London
2018-01-01 09:02:00,3,London
2018-01-01 09:03:00,3,London
2018-01-01 09:04:00,3,London
...,...,...
2018-12-31 15:56:00,4,Cambridge
2018-12-31 15:57:00,2,Cambridge
2018-12-31 15:58:00,3,Cambridge


In [57]:
df.loc['2018','num'].sum()

1231190

In [58]:
df['2018'].groupby('city').sum()

/var/folders/0n/nb074wzd4kl1nmdvkbdnpgym0000gn/T/ipykernel_11495/1368259670.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df['2018'].groupby('city').sum()


,num
city,
Cambridge,308428
Durham,307965
London,307431
Oxford,307366


### 8. Select data with a specific month or a specific day of the month

In [59]:
df.loc['2018-5']

,num,city
date,,
2018-05-01 09:00:00,1,London
2018-05-01 09:01:00,4,London
2018-05-01 09:02:00,3,London
2018-05-01 09:03:00,2,London
2018-05-01 09:04:00,3,London
...,...,...
2018-05-31 15:56:00,3,Cambridge
2018-05-31 15:57:00,4,Cambridge
2018-05-31 15:58:00,2,Cambridge


In [60]:
df.loc['2018-5-1']

,num,city
date,,
2018-05-01 09:00:00,1,London
2018-05-01 09:01:00,4,London
2018-05-01 09:02:00,3,London
2018-05-01 09:03:00,2,London
2018-05-01 09:04:00,3,London
...,...,...
2018-05-01 15:56:00,2,Cambridge
2018-05-01 15:57:00,3,Cambridge
2018-05-01 15:58:00,3,Cambridge


In [65]:
cond = df.index.month==3

In [66]:
cond

array([False, False, False, ..., False, False, False])

In [67]:
df[cond]

,num,city
date,,
2015-03-02 09:00:00,1,London
2015-03-02 09:01:00,3,London
2015-03-02 09:02:00,2,London
2015-03-02 09:03:00,3,London
2015-03-02 09:04:00,1,London
...,...,...
2018-03-30 15:56:00,4,Cambridge
2018-03-30 15:57:00,1,Cambridge
2018-03-30 15:58:00,3,Cambridge


In [61]:
cond = df.index.month==2
df[cond]

,num,city
date,,
2015-02-02 09:00:00,4,London
2015-02-02 09:01:00,3,London
2015-02-02 09:02:00,3,London
2015-02-02 09:03:00,2,London
2015-02-02 09:04:00,1,London
...,...,...
2018-02-28 15:56:00,4,Cambridge
2018-02-28 15:57:00,3,Cambridge
2018-02-28 15:58:00,4,Cambridge


### 9. Select data between two dates

In [62]:
df.loc['2016' : '2018']

/var/folders/0n/nb074wzd4kl1nmdvkbdnpgym0000gn/T/ipykernel_11495/380412996.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df.loc['2016' : '2018']


,num,city
date,,
2016-01-01 09:00:00,4,London
2016-01-01 09:01:00,3,London
2016-01-01 09:02:00,4,London
2016-01-01 09:03:00,4,London
2016-01-01 09:04:00,2,London
...,...,...
2018-12-31 15:56:00,4,Cambridge
2018-12-31 15:57:00,2,Cambridge
2018-12-31 15:58:00,3,Cambridge


In [63]:
df.loc['2018-5-2 10' : '2018-5-2 11' ]

,num,city
date,,
2018-05-02 10:00:00,2,London
2018-05-02 10:01:00,3,London
2018-05-02 10:02:00,4,London
2018-05-02 10:03:00,4,London
2018-05-02 10:04:00,4,London
...,...,...
2018-05-02 11:55:00,3,Cambridge
2018-05-02 11:56:00,3,Cambridge
2018-05-02 11:57:00,4,Cambridge


In [ ]:
df.loc['2018-5-2 10:30' : '2018-5-2 10:45' ]

In [ ]:
df.between_time('10:30','10:45')

In [ ]:
df['rolling_sum_backfilled'] = df['rolling_sum'].fillna(method='backfill')
df.head()